In [ ]:
!pip install pandas
!pip install scikit-learn

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [ ]:
import os
import re
import numpy as np
import pandas as pd
import sklearn
from sklearn.utils import shuffle

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

from nltk.tokenize import word_tokenize
from scipy.sparse import csr_matrix
from collections import Counter

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Embedding,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D,Input,Conv1D,MaxPooling1D,Flatten
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.initializers import Constant
from sklearn.utils import class_weight
import tensorflow as tf

import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import string
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
df = pd.read_csv("data/OLIDv1/olid-training-v1.0.tsv", sep='\t')
df.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [ ]:
# Reference: https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string
CLEANR = re.compile('<.*?>') 
def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

def preprocess(sent):
    alphabet = "abcdefghijklmnopqrstuvwxyz 0123456789',."
    sent = sent.lower() 
    sent = cleanhtml(sent) # remove html tags 
    cleaned_sent_list = [char if char in alphabet else ' ' for char in sent] # remove all tags not in the alphabet
    cleaned_sent = ''.join(cleaned_sent_list)
    cleaned_sent = cleaned_sent.replace("n't",' not') # replace words like "isn't" with "is not"
    cleaned_sent = ' . '.join([x for x in cleaned_sent.split('.') if len(x)>0]) # remove multiple periods, and add spaces before and after a period
    cleaned_sent = ' , '.join([x for x in cleaned_sent.split(',') if len(x)>0]) # add spaces before and after a comma
    cleaned_sent = ' '.join(cleaned_sent.split()) # remove multiple spaces
    return cleaned_sent

def print_metrics(y,y_p):
    print('Accuracy:',accuracy_score(y,y_p))
    print('Precision:',precision_score(y,y_p))
    print('Recall:',recall_score(y,y_p))
    print('F1 score:',f1_score(y,y_p))

## Training

In [ ]:
x = df.tweet
y = df.subtask_a.apply(lambda x: 1 if x=='OFF' else 0)

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
data_train = [preprocess(tweet) for tweet in x_train]
data_val = [preprocess(tweet) for tweet in x_val]

n_features = 500
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3),max_features=n_features)
X_train = vectorizer.fit_transform(data_train)
X_val = vectorizer.transform(data_val)

In [ ]:
X_train = X_train.toarray().reshape([-1,n_features,1])
X_val = X_val.toarray().reshape([-1,n_features,1])

In [ ]:
def model_cnn(n_features=500):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=5, activation='relu', input_shape=(n_features,1)))
    model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=5, activation='relu'))
    model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=5, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model = model_cnn(500)
history = model.fit(X_train.reshape([-1,n_features,1]),y_train,
                    validation_data=(X_val.reshape([-1,n_features,1]),y_val),batch_size=256,epochs=25)

2021-12-06 18:09:04.794786: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-06 18:09:04.806050: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-06 18:09:04.807039: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-06 18:09:04.809221: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-06 18:09:04.810171: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read f

Epoch 1/25


2021-12-06 18:09:07.779519: I tensorflow/stream_executor/cuda/cuda_dnn.cc:381] Loaded cuDNN version 8204


37/37 [==============================] - 4s 15ms/step - loss: 0.6412 - accuracy: 0.6644 - val_loss: 0.6308 - val_accuracy: 0.6644
Epoch 2/25
37/37 [==============================] - 0s 7ms/step - loss: 0.6173 - accuracy: 0.6713 - val_loss: 0.6053 - val_accuracy: 0.6805
Epoch 3/25
37/37 [==============================] - 0s 7ms/step - loss: 0.5879 - accuracy: 0.7074 - val_loss: 0.5757 - val_accuracy: 0.7183
Epoch 4/25
37/37 [==============================] - 0s 7ms/step - loss: 0.5599 - accuracy: 0.7265 - val_loss: 0.5812 - val_accuracy: 0.7195
Epoch 5/25
37/37 [==============================] - 0s 6ms/step - loss: 0.5459 - accuracy: 0.7352 - val_loss: 0.5675 - val_accuracy: 0.7246
Epoch 6/25
37/37 [==============================] - 0s 6ms/step - loss: 0.5347 - accuracy: 0.7394 - val_loss: 0.5610 - val_accuracy: 0.7243
Epoch 7/25
37/37 [==============================] - 0s 6ms/step - loss: 0.5283 - accuracy: 0.7414 - val_loss: 0.5682 - val_accuracy: 0.7271
Epoch 8/25
37/37 [============

In [ ]:
y_val_p = model.predict(X_val)
y_val_p = np.round(y_val_p).flatten()
print(classification_report(y_val, y_val_p))

              precision    recall  f1-score   support

           0       0.74      0.79      0.76      2639
           1       0.52      0.45      0.48      1333

    accuracy                           0.67      3972
   macro avg       0.63      0.62      0.62      3972
weighted avg       0.66      0.67      0.67      3972



In [ ]:
print_metrics(y_val, y_val_p)

Accuracy: 0.6739677744209466
Precision: 0.5164644714038128
Recall: 0.44711177794448614
F1 score: 0.4792923200643346


## Testing 

In [ ]:
df_test = pd.read_csv("data/OLIDv1/testset-levela.tsv", sep='\t').set_index('id')
y_test = pd.read_csv("data/OLIDv1/labels-levela.csv", names = ['id','subtask_a']).set_index('id')
df_test = df_test.join(y_test).reset_index()

In [ ]:
x_test = df_test.tweet
y_test = df_test.subtask_a.apply(lambda x: 1 if x=='OFF' else 0)

In [ ]:
data_test = [preprocess(tweet) for tweet in x_test]
X_test = vectorizer.transform(data_test)
X_test = X_test.toarray().reshape([-1,n_features,1])
y_test_p = model.predict(X_test)
y_test_p = np.round(y_test_p).flatten()

In [ ]:
print(classification_report(y_test, y_test_p))

              precision    recall  f1-score   support

           0       0.79      0.83      0.81       620
           1       0.48      0.42      0.45       240

    accuracy                           0.71       860
   macro avg       0.63      0.62      0.63       860
weighted avg       0.70      0.71      0.71       860

